In [1]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from math import exp

In [2]:
X_train = ["new york is a hell of a town",
                    "new york was originally dutch",
                    "the big apple is great",
                    "new york is also called the big apple",
                    "nyc is nice",
                    "people abbreviate new york city as nyc",
                    "the capital of great britain is london",
                    "london is in the uk",
                    "london is in england",
                    "london is in great britain",
                    "it rains a lot in london",
                    "london hosts the british museum",
                    "new york is great and so is london",
                    "i like london better than new york"]
y_train = [['New York'],['New York'],['New York'],['New York'],['New York'],['New York'],['London'],['London'],['London'],['London'],['London'],['London'],['New York','London'],['New York','London']]

mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(y_train)

X_test = ['nice day in nyc',
                   'welcome to london',
                   'hello welcome to new york. enjoy it here and london too']
target_names = ['New York', 'London']

classifier = Pipeline([
    ('vectorizer', CountVectorizer(ngram_range=(1, 2), stop_words='english', max_features=50000)),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(LinearSVC(verbose=1, max_iter=100), n_jobs=6))])

classifier.fit(X_train, y_train)
predicted = classifier.predict(X_test)
for item, labels in zip(X_test, mlb.inverse_transform(predicted)):
    print('%s => %s' % (item, labels))


to_prob = np.vectorize(lambda x: 1 / (1 + exp(-x)))
print(to_prob(classifier.decision_function(X_test)))

nice day in nyc => ('New York',)
welcome to london => ('London',)
hello welcome to new york. enjoy it here and london too => ('London', 'New York')
[[ 0.36310356  0.64518675]
 [ 0.74537813  0.3457724 ]
 [ 0.54703065  0.6613977 ]]
